In [1]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot

# Example cache directory path, it determines where downloaded data will be stored
output_dir = './ecephys_cache_dir/'

manifest_path = os.path.join(output_dir, "manifest.json")

cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

pd.set_option('display.max_columns', None)

In [2]:
session_id = 721123822 # 797828357
session = cache.get_session_data(session_id)

In [3]:
VI_structure_acronyms = [x for x in cache.get_structure_acronyms() if x is not np.nan and 'VI' in x]
VI_structure_acronyms

['VISrl',
 'VIS',
 'VISp',
 'VISpm',
 'VISam',
 'VISal',
 'VISl',
 'VISmma',
 'VISmmp',
 'VISli']

In [4]:
probes = cache.get_probes()
probes = probes[probes['ecephys_session_id'] == session_id]

for i, x in probes.iterrows():
    print(f'probe id: {i: d}')
    print('structure: ' + ', '.join([s for s in x['ecephys_structure_acronyms'] if s in VI_structure_acronyms]))

probe id:  760213137
structure: VISam
probe id:  760213142
structure: VIS
probe id:  760213145
structure: VISp
probe id:  760213147
structure: VISl
probe id:  760213150
structure: VISal
probe id:  760213153
structure: VISrl


In [5]:
VI_units = session.units.loc[[i for i, x in session.units.iterrows() if x['ecephys_structure_acronym'] in VI_structure_acronyms]]
len(VI_units)

214

In [6]:
units = cache.get_units(amplitude_cutoff_maximum = np.inf,
                        presence_ratio_minimum = -np.inf,
                        isi_violations_maximum = np.inf)
units = units[units['ecephys_session_id'] == session_id]

VI_units_id = {i for i, x in session.units.iterrows() if x['ecephys_structure_acronym'] in VI_structure_acronyms}
VI_units = session.units.loc[[i for i in session.units.index if i in VI_units_id]]
len(VI_units)

214

In [7]:
ecephys_structure_acronyms = 'VISp'
sel_units = VI_units[VI_units['ecephys_structure_acronym'] == ecephys_structure_acronyms]
# sel_units = VI_units

In [8]:
ccf_coords = ['anterior_posterior_ccf_coordinate', 'dorsal_ventral_ccf_coordinate', 'left_right_ccf_coordinate']
units_coord = sel_units[ccf_coords].values
units_coord_mean = units_coord.mean(axis=0)
center_unit_id = sel_units.index[np.argmin(np.sum((units_coord - units_coord_mean) ** 2, axis=1))]

channel_index = sel_units.loc[center_unit_id, 'probe_channel_number']
probe_id = sel_units.loc[center_unit_id, 'probe_id']

channel = session.channels[(session.channels.probe_channel_number == channel_index) & 
                           (session.channels.probe_id == probe_id)]
channel

,filtering,probe_channel_number,probe_horizontal_position,probe_id,probe_vertical_position,structure_acronym,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate
id,,,,,,,,,,,
850245985,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,234,59,760213145,2360,VISp,385.0,VISp,8650.0,1130.0,8035.0


In [10]:
channels_coord = session.channels[ccf_coords].values
center_channel_id = session.channels.index[np.argmin(np.sum((channels_coord - units_coord_mean) ** 2, axis=1))]
session.channels.loc[[center_channel_id]]

,filtering,probe_channel_number,probe_horizontal_position,probe_id,probe_vertical_position,structure_acronym,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate
id,,,,,,,,,,,
850245983,AP band: 500 Hz high-pass; LFP band: 1000 Hz l...,233,11,760213145,2340,VISp,385.0,VISp,8646.0,1141.0,8035.0


In [9]:
stimulus_presentations = session.stimulus_presentations

In [10]:
drifting_gratings_presentations = stimulus_presentations.loc[(stimulus_presentations['stimulus_name'] == 'drifting_gratings')]

In [11]:
drifting_gratings_presentations

,color,contrast,frame,orientation,phase,size,spatial_frequency,start_time,stimulus_block,stimulus_name,stop_time,temporal_frequency,x_position,y_position,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,,,,,
3798,null,0.8,null,315.0,"[21211.93333333, 21211.93333333]","[250.0, 250.0]",0.04,1587.522722,2.0,drifting_gratings,1589.524372,15.0,null,null,2.00165,246
3799,null,0.8,null,0.0,"[21211.93333333, 21211.93333333]","[250.0, 250.0]",0.04,1590.525218,2.0,drifting_gratings,1592.526888,4.0,null,null,2.00167,247
3800,null,0.8,null,315.0,"[21211.93333333, 21211.93333333]","[250.0, 250.0]",0.04,1593.527732,2.0,drifting_gratings,1595.529392,4.0,null,null,2.00166,248
3801,null,0.8,null,90.0,"[21211.93333333, 21211.93333333]","[250.0, 250.0]",0.04,1596.530228,2.0,drifting_gratings,1598.531898,15.0,null,null,2.00167,249
3802,null,0.8,null,270.0,"[21211.93333333, 21211.93333333]","[250.0, 250.0]",0.04,1599.532762,2.0,drifting_gratings,1601.534412,15.0,null,null,2.00165,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49426,null,0.8,null,45.0,"[21211.93333333, 21211.93333333]","[250.0, 250.0]",0.04,5379.689398,7.0,drifting_gratings,5381.691068,2.0,null,null,2.00167,284
49427,null,0.8,null,180.0,"[21211.93333333, 21211.93333333]","[250.0, 250.0]",0.04,5382.691922,7.0,drifting_gratings,5384.693572,2.0,null,null,2.00165,285
49428,null,0.8,null,90.0,"[21211.93333333, 21211.93333333]","[250.0, 250.0]",0.04,5385.694408,7.0,drifting_gratings,5387.696088,1.0,null,null,2.00168,269


In [12]:
len(drifting_gratings_presentations)

628

In [13]:
help(session.presentationwise_spike_counts)

Help on method presentationwise_spike_counts in module allensdk.brain_observatory.ecephys.ecephys_session:

presentationwise_spike_counts(bin_edges, stimulus_presentation_ids, unit_ids, binarize=False, dtype=None, large_bin_size_threshold=0.001, time_domain_callback=None) method of allensdk.brain_observatory.ecephys.ecephys_session.EcephysSession instance
    Build an array of spike counts surrounding stimulus onset per
    unit and stimulus frame.
    
    Parameters
    ---------
    bin_edges : numpy.ndarray
        Spikes will be counted into the bins defined by these edges.
        Values are in seconds, relative to stimulus onset.
    stimulus_presentation_ids : array-like
        Filter to these stimulus presentations
    unit_ids : array-like
        Filter to these units
    binarize : bool, optional
        If true, all counts greater than 0 will be treated as 1. This
        results in lower storage overhead, but is only reasonable if bin
        sizes are fine (<= 1 millise